#### Imports

In [1]:
import json

from nltk import CFG
from nltk.parse.generate import generate

#### Read in files

In [2]:
# change paths if needed
with open('grammar.txt', 'r', encoding='utf-8') as file:
    rules = file.read().split('Production rules for 1st item utterances\n')[1]
model = json.load(open('interaction_model.json', 'r', encoding='utf-8'))

In [3]:
print(rules)

S -> O | O L | P O | P O L | L O | L P O
O -> object | article object | adjective object | article adjective object | particle adjective object | particle article adjective object | particle article object
L -> L L | L image | particle L | location | location_transitive O | location_ditransitive O 'und' O
P -> verb



### Create grammar
- extract slots from interaction model
- create grammar from production rules

In [4]:
# create a dictionary from interaction model slots {slot: list of slot values and synonyms}
slots = dict()
for s in model["interactionModel"]["languageModel"]["types"]:
    slots[s['name']] = list()
    for v in s['values']:
        if 'synonyms' in v['name']:
            slots[s['name']] += v['name']['synonyms']
        slots[s['name']] += [v['name']['value']]
    # replace ' by `
    for slot, index in zip(slots[s['name']], range(len(slots[s['name']]))):
        slots[s['name']][index] = slot.replace('\'', '`')

In [5]:
# TODO recreate grammar from intent
intent = 'ItemsIntent'
model["interactionModel"]["languageModel"]["intents"][intent]

TypeError: list indices must be integers or slices, not str

### Slots from Items Dict

In [ ]:
g = """
S -> O | O L | P O | P O L | L O | L P O
O -> object | article object | adjective object | article adjective object | particle adjective object | particle article adjective object | particle article object
L -> L L | L image | particle L | location | location_transitive O | location_ditransitive O 'und' O
P -> verb
"""

In [ ]:
items_dict = json.loads('items_dict.json')

for item in items:
    generate(item)

In [ ]:
def generate(item, totxt=True):
    utterances = []
    
    if totxt:
        with open(f'utterances/{item}.txt', 'r', encoding='utf-8') as file:
            for u in utterances:
                file.write(u + '\n')

In [6]:
# add terminal rules extracted from slots
for s, v in slots.items():
    values = ' | '.join([f'\'{val}\'' for val in v])
    rule = f'{s} -> {values}\n'
    rules += rule

In [7]:
print(rules)

S -> O | O L | P O | P O L | L O | L P O
O -> object | article object | adjective object | article adjective object | particle adjective object | particle article adjective object | particle article object
L -> L L | L image | particle L | location | location_transitive O | location_ditransitive O 'und' O
P -> verb
article -> 'wat' | 'was' | 'etwas' | 'dat' | 'des' | 'dor' | 'ditt' | 'dit' | 'diesem' | 'diesen' | 'dieses' | 'diese' | 'dieser' | 'so eine' | 'so einen' | 'so ein' | 'eine' | 'einen' | 'einem' | 'ein' | 'dem' | 'den' | 'das' | 'die' | 'der'
object -> 'Absolvent' | 'Student' | 'Gesicht' | 'Smiley' | 'Sterne' | 'Stern' | 'Bildschirm' | 'Podest' | 'Binärcode' | 'Stühle' | 'Stuhl' | 'Seeigel' | 'Mine' | 'Pfeile' | 'Pfeil' | 'Legobaustein' | 'Legostein' | 'Tisch' | 'ABC' | 'Molekül' | 'Atom' | 'Schrank' | 'Delfin' | 'Medaille' | 'Tauben' | 'Taube' | 'Vögel' | 'Vogel' | 'Pfeile' | 'Pfeil' | 'Regenbogen' | 'Reagenzglas' | 'Schultasche' | 'Säulen' | 'Spinne' | 'Wolken' | 'Wolke' |

In [8]:
grammar = CFG.fromstring(rules)
grammar

<Grammar with 331 productions>

In [9]:
grammar.productions()

[S -> O,
 S -> O L,
 S -> P O,
 S -> P O L,
 S -> L O,
 S -> L P O,
 O -> object,
 O -> article object,
 O -> adjective object,
 O -> article adjective object,
 O -> particle adjective object,
 O -> particle article adjective object,
 O -> particle article object,
 L -> L L,
 L -> L image,
 L -> particle L,
 L -> location,
 L -> location_transitive O,
 L -> location_ditransitive O 'und' O,
 P -> verb,
 article -> 'wat',
 article -> 'was',
 article -> 'etwas',
 article -> 'dat',
 article -> 'des',
 article -> 'dor',
 article -> 'ditt',
 article -> 'dit',
 article -> 'diesem',
 article -> 'diesen',
 article -> 'dieses',
 article -> 'diese',
 article -> 'dieser',
 article -> 'so eine',
 article -> 'so einen',
 article -> 'so ein',
 article -> 'eine',
 article -> 'einen',
 article -> 'einem',
 article -> 'ein',
 article -> 'dem',
 article -> 'den',
 article -> 'das',
 article -> 'die',
 article -> 'der',
 object -> 'Absolvent',
 object -> 'Student',
 object -> 'Gesicht',
 object -> 'Smiley

### Generate utterances

In [10]:
def sentences_per_depth(g):
    for i in range(10):
        print('-'*50 + f'\nDEPTH{i}')
        for sentence in generate(g, depth=i):
            print(' '.join(sentence))

In [11]:
def sent_dep(g, d):
    for sentence in generate(g, depth=d):
            print(' '.join(sentence))

In [12]:
sent_dep(grammar, 5)

Absolvent
Student
Gesicht
Smiley
Sterne
Stern
Bildschirm
Podest
Binärcode
Stühle
Stuhl
Seeigel
Mine
Pfeile
Pfeil
Legobaustein
Legostein
Tisch
ABC
Molekül
Atom
Schrank
Delfin
Medaille
Tauben
Taube
Vögel
Vogel
Pfeile
Pfeil
Regenbogen
Reagenzglas
Schultasche
Säulen
Spinne
Wolken
Wolke
Blumentopf
Pflanze
Treppe
Lineal
Büro
Schreibtisch
Giraffenkopf
Giraffe
Zahnrad
other
Sachen
Sache
Teile
Gegenstände
Dinger
Dingens
Dings
Objekt
Gegenstand
Teil
Ding
Uhr
Buchstabenwürfel
Würfel
Programmfenster
Bildschirm
Computerbildschirm
Konsole
Terminal
Fernrohr
Teleskop
Schultafel
Tafel
Schreibutensil
Buntstift
Stift
Gameboy
Switch
Spielekonsole
Spielkonsole
Spaceshuttle
Flugzeug
Raumschiff
Saturn
Planet
Ärztin
Arzt
Ingenieurin
Ingenieur
Mann
Frau
Wissenschaftlerin
Wissenschaftler
Forscherin
Forscher
Personen
Mensch
Person
Teleskop
Mikroskop
Kasten
Lab
Labor
Grafiken
Diagramm
Wellen
Kurven
Birne
Glühbirne
Screen
Gleichung
Demo
Klimaaktivistinnen
Klimaaktivisten
Fridays for Future
FFF
Kinder mit Flagge
Fr

dor Binärcode
dor Stühle
dor Stuhl
dor Seeigel
dor Mine
dor Pfeile
dor Pfeil
dor Legobaustein
dor Legostein
dor Tisch
dor ABC
dor Molekül
dor Atom
dor Schrank
dor Delfin
dor Medaille
dor Tauben
dor Taube
dor Vögel
dor Vogel
dor Pfeile
dor Pfeil
dor Regenbogen
dor Reagenzglas
dor Schultasche
dor Säulen
dor Spinne
dor Wolken
dor Wolke
dor Blumentopf
dor Pflanze
dor Treppe
dor Lineal
dor Büro
dor Schreibtisch
dor Giraffenkopf
dor Giraffe
dor Zahnrad
dor other
dor Sachen
dor Sache
dor Teile
dor Gegenstände
dor Dinger
dor Dingens
dor Dings
dor Objekt
dor Gegenstand
dor Teil
dor Ding
dor Uhr
dor Buchstabenwürfel
dor Würfel
dor Programmfenster
dor Bildschirm
dor Computerbildschirm
dor Konsole
dor Terminal
dor Fernrohr
dor Teleskop
dor Schultafel
dor Tafel
dor Schreibutensil
dor Buntstift
dor Stift
dor Gameboy
dor Switch
dor Spielekonsole
dor Spielkonsole
dor Spaceshuttle
dor Flugzeug
dor Raumschiff
dor Saturn
dor Planet
dor Ärztin
dor Arzt
dor Ingenieurin
dor Ingenieur
dor Mann
dor Frau
dor W

dieser Sache
dieser Teile
dieser Gegenstände
dieser Dinger
dieser Dingens
dieser Dings
dieser Objekt
dieser Gegenstand
dieser Teil
dieser Ding
dieser Uhr
dieser Buchstabenwürfel
dieser Würfel
dieser Programmfenster
dieser Bildschirm
dieser Computerbildschirm
dieser Konsole
dieser Terminal
dieser Fernrohr
dieser Teleskop
dieser Schultafel
dieser Tafel
dieser Schreibutensil
dieser Buntstift
dieser Stift
dieser Gameboy
dieser Switch
dieser Spielekonsole
dieser Spielkonsole
dieser Spaceshuttle
dieser Flugzeug
dieser Raumschiff
dieser Saturn
dieser Planet
dieser Ärztin
dieser Arzt
dieser Ingenieurin
dieser Ingenieur
dieser Mann
dieser Frau
dieser Wissenschaftlerin
dieser Wissenschaftler
dieser Forscherin
dieser Forscher
dieser Personen
dieser Mensch
dieser Person
dieser Teleskop
dieser Mikroskop
dieser Kasten
dieser Lab
dieser Labor
dieser Grafiken
dieser Diagramm
dieser Wellen
dieser Kurven
dieser Birne
dieser Glühbirne
dieser Screen
dieser Gleichung
dieser Demo
dieser Klimaaktivistinnen
d

ein Sterne
ein Stern
ein Bildschirm
ein Podest
ein Binärcode
ein Stühle
ein Stuhl
ein Seeigel
ein Mine
ein Pfeile
ein Pfeil
ein Legobaustein
ein Legostein
ein Tisch
ein ABC
ein Molekül
ein Atom
ein Schrank
ein Delfin
ein Medaille
ein Tauben
ein Taube
ein Vögel
ein Vogel
ein Pfeile
ein Pfeil
ein Regenbogen
ein Reagenzglas
ein Schultasche
ein Säulen
ein Spinne
ein Wolken
ein Wolke
ein Blumentopf
ein Pflanze
ein Treppe
ein Lineal
ein Büro
ein Schreibtisch
ein Giraffenkopf
ein Giraffe
ein Zahnrad
ein other
ein Sachen
ein Sache
ein Teile
ein Gegenstände
ein Dinger
ein Dingens
ein Dings
ein Objekt
ein Gegenstand
ein Teil
ein Ding
ein Uhr
ein Buchstabenwürfel
ein Würfel
ein Programmfenster
ein Bildschirm
ein Computerbildschirm
ein Konsole
ein Terminal
ein Fernrohr
ein Teleskop
ein Schultafel
ein Tafel
ein Schreibutensil
ein Buntstift
ein Stift
ein Gameboy
ein Switch
ein Spielekonsole
ein Spielkonsole
ein Spaceshuttle
ein Flugzeug
ein Raumschiff
ein Saturn
ein Planet
ein Ärztin
ein Arzt
ein In

ähnlich Gesicht
ähnlich Smiley
ähnlich Sterne
ähnlich Stern
ähnlich Bildschirm
ähnlich Podest
ähnlich Binärcode
ähnlich Stühle
ähnlich Stuhl
ähnlich Seeigel
ähnlich Mine
ähnlich Pfeile
ähnlich Pfeil
ähnlich Legobaustein
ähnlich Legostein
ähnlich Tisch
ähnlich ABC
ähnlich Molekül
ähnlich Atom
ähnlich Schrank
ähnlich Delfin
ähnlich Medaille
ähnlich Tauben
ähnlich Taube
ähnlich Vögel
ähnlich Vogel
ähnlich Pfeile
ähnlich Pfeil
ähnlich Regenbogen
ähnlich Reagenzglas
ähnlich Schultasche
ähnlich Säulen
ähnlich Spinne
ähnlich Wolken
ähnlich Wolke
ähnlich Blumentopf
ähnlich Pflanze
ähnlich Treppe
ähnlich Lineal
ähnlich Büro
ähnlich Schreibtisch
ähnlich Giraffenkopf
ähnlich Giraffe
ähnlich Zahnrad
ähnlich other
ähnlich Sachen
ähnlich Sache
ähnlich Teile
ähnlich Gegenstände
ähnlich Dinger
ähnlich Dingens
ähnlich Dings
ähnlich Objekt
ähnlich Gegenstand
ähnlich Teil
ähnlich Ding
ähnlich Uhr
ähnlich Buchstabenwürfel
ähnlich Würfel
ähnlich Programmfenster
ähnlich Bildschirm
ähnlich Computerbildschirm

...

wat drei Raumfahrerinnen
wat drei Frauen
wat drei Astronautinnen
wat drei Luftballon
wat drei Ball
wat drei Ballon
wat drei Tier
wat drei Insekt
wat drei Biene
wat drei Weltkugel
wat drei Erdkugel
wat drei Globus
wat gelbes Absolvent
wat gelbes Student
wat gelbes Gesicht
wat gelbes Smiley
wat gelbes Sterne
wat gelbes Stern
wat gelbes Bildschirm
wat gelbes Podest
wat gelbes Binärcode
wat gelbes Stühle
wat gelbes Stuhl
wat gelbes Seeigel
wat gelbes Mine
wat gelbes Pfeile
wat gelbes Pfeil
wat gelbes Legobaustein
wat gelbes Legostein
wat gelbes Tisch
wat gelbes ABC
wat gelbes Molekül
wat gelbes Atom
wat gelbes Schrank
wat gelbes Delfin
wat gelbes Medaille
wat gelbes Tauben
wat gelbes Taube
wat gelbes Vögel
wat gelbes Vogel
wat gelbes Pfeile
wat gelbes Pfeil
wat gelbes Regenbogen
wat gelbes Reagenzglas
wat gelbes Schultasche
wat gelbes Säulen
wat gelbes Spinne
wat gelbes Wolken
wat gelbes Wolke
wat gelbes Blumentopf
wat gelbes Pflanze
wat gelbes Treppe
wat gelbes Lineal
wat gelbes Büro
wat 

wat grünen Buchstabenwürfel
wat grünen Würfel
wat grünen Programmfenster
wat grünen Bildschirm
wat grünen Computerbildschirm
wat grünen Konsole
wat grünen Terminal
wat grünen Fernrohr
wat grünen Teleskop
wat grünen Schultafel
wat grünen Tafel
wat grünen Schreibutensil
wat grünen Buntstift
wat grünen Stift
wat grünen Gameboy
wat grünen Switch
wat grünen Spielekonsole
wat grünen Spielkonsole
wat grünen Spaceshuttle
wat grünen Flugzeug
wat grünen Raumschiff
wat grünen Saturn
wat grünen Planet
wat grünen Ärztin
wat grünen Arzt
wat grünen Ingenieurin
wat grünen Ingenieur
wat grünen Mann
wat grünen Frau
wat grünen Wissenschaftlerin
wat grünen Wissenschaftler
wat grünen Forscherin
wat grünen Forscher
wat grünen Personen
wat grünen Mensch
wat grünen Person
wat grünen Teleskop
wat grünen Mikroskop
wat grünen Kasten
wat grünen Lab
wat grünen Labor
wat grünen Grafiken
wat grünen Diagramm
wat grünen Wellen
wat grünen Kurven
wat grünen Birne
wat grünen Glühbirne
wat grünen Screen
wat grünen Gleichu

wat große Boot
wat große Drillinge
wat große Raumfahrerinnen
wat große Frauen
wat große Astronautinnen
wat große Luftballon
wat große Ball
wat große Ballon
wat große Tier
wat große Insekt
wat große Biene
wat große Weltkugel
wat große Erdkugel
wat große Globus
wat groß Absolvent
wat groß Student
wat groß Gesicht
wat groß Smiley
wat groß Sterne
wat groß Stern
wat groß Bildschirm
wat groß Podest
wat groß Binärcode
wat groß Stühle
wat groß Stuhl
wat groß Seeigel
wat groß Mine
wat groß Pfeile
wat groß Pfeil
wat groß Legobaustein
wat groß Legostein
wat groß Tisch
wat groß ABC
wat groß Molekül
wat groß Atom
wat groß Schrank
wat groß Delfin
wat groß Medaille
wat groß Tauben
wat groß Taube
wat groß Vögel
wat groß Vogel
wat groß Pfeile
wat groß Pfeil
wat groß Regenbogen
wat groß Reagenzglas
wat groß Schultasche
wat groß Säulen
wat groß Spinne
wat groß Wolken
wat groß Wolke
wat groß Blumentopf
wat groß Pflanze
wat groß Treppe
wat groß Lineal
wat groß Büro
wat groß Schreibtisch
wat groß Giraffenko

was ähnlich Buchstabenwürfel
was ähnlich Würfel
was ähnlich Programmfenster
was ähnlich Bildschirm
was ähnlich Computerbildschirm
was ähnlich Konsole
was ähnlich Terminal
was ähnlich Fernrohr
was ähnlich Teleskop
was ähnlich Schultafel
was ähnlich Tafel
was ähnlich Schreibutensil
was ähnlich Buntstift
was ähnlich Stift
was ähnlich Gameboy
was ähnlich Switch
was ähnlich Spielekonsole
was ähnlich Spielkonsole
was ähnlich Spaceshuttle
was ähnlich Flugzeug
was ähnlich Raumschiff
was ähnlich Saturn
was ähnlich Planet
was ähnlich Ärztin
was ähnlich Arzt
was ähnlich Ingenieurin
was ähnlich Ingenieur
was ähnlich Mann
was ähnlich Frau
was ähnlich Wissenschaftlerin
was ähnlich Wissenschaftler
was ähnlich Forscherin
was ähnlich Forscher
was ähnlich Personen
was ähnlich Mensch
was ähnlich Person
was ähnlich Teleskop
was ähnlich Mikroskop
was ähnlich Kasten
was ähnlich Lab
was ähnlich Labor
was ähnlich Grafiken
was ähnlich Diagramm
was ähnlich Wellen
was ähnlich Kurven
was ähnlich Birne
was ähnlich

KeyboardInterrupt: 

In [13]:
# generate sentences for a certain object, e.g. 'Reagenzglas'
for sentence in generate(grammar, depth=7, n=1000000):
    if len(sentence) >= 4 and 'Reagenzglas' in sentence:
        print(' '.join(sentence))

schräg wat ähnliche Reagenzglas
schräg wat ähnlich Reagenzglas
schräg wat bunte Reagenzglas
schräg wat bunten Reagenzglas
schräg wat bunt Reagenzglas
schräg wat schwarze Reagenzglas
schräg wat schwarzen Reagenzglas
schräg wat schwarz Reagenzglas
schräg wat weißen Reagenzglas
schräg wat weiße Reagenzglas
schräg wat lächelnde Reagenzglas
schräg wat lächelnd Reagenzglas
schräg wat blaue Reagenzglas
schräg wat blau Reagenzglas
schräg wat roten Reagenzglas
schräg wat rotes Reagenzglas
schräg wat rote Reagenzglas
schräg wat rot Reagenzglas
schräg wat zwei Reagenzglas
schräg wat drei Reagenzglas
schräg wat gelbes Reagenzglas
schräg wat gelbe Reagenzglas
schräg wat gelb Reagenzglas
schräg wat grünen Reagenzglas
schräg wat grüne Reagenzglas
schräg wat grün Reagenzglas
schräg wat große Reagenzglas
schräg wat groß Reagenzglas
schräg wat kleine Reagenzglas
schräg wat klein Reagenzglas
schräg was ähnliche Reagenzglas
schräg was ähnlich Reagenzglas
schräg was bunte Reagenzglas
schräg was bunten Reag

schräg diese blau Reagenzglas
schräg diese roten Reagenzglas
schräg diese rotes Reagenzglas
schräg diese rote Reagenzglas
schräg diese rot Reagenzglas
schräg diese zwei Reagenzglas
schräg diese drei Reagenzglas
schräg diese gelbes Reagenzglas
schräg diese gelbe Reagenzglas
schräg diese gelb Reagenzglas
schräg diese grünen Reagenzglas
schräg diese grüne Reagenzglas
schräg diese grün Reagenzglas
schräg diese große Reagenzglas
schräg diese groß Reagenzglas
schräg diese kleine Reagenzglas
schräg diese klein Reagenzglas
schräg dieser ähnliche Reagenzglas
schräg dieser ähnlich Reagenzglas
schräg dieser bunte Reagenzglas
schräg dieser bunten Reagenzglas
schräg dieser bunt Reagenzglas
schräg dieser schwarze Reagenzglas
schräg dieser schwarzen Reagenzglas
schräg dieser schwarz Reagenzglas
schräg dieser weißen Reagenzglas
schräg dieser weiße Reagenzglas
schräg dieser lächelnde Reagenzglas
schräg dieser lächelnd Reagenzglas
schräg dieser blaue Reagenzglas
schräg dieser blau Reagenzglas
schräg die

schräg das schwarz Reagenzglas
schräg das weißen Reagenzglas
schräg das weiße Reagenzglas
schräg das lächelnde Reagenzglas
schräg das lächelnd Reagenzglas
schräg das blaue Reagenzglas
schräg das blau Reagenzglas
schräg das roten Reagenzglas
schräg das rotes Reagenzglas
schräg das rote Reagenzglas
schräg das rot Reagenzglas
schräg das zwei Reagenzglas
schräg das drei Reagenzglas
schräg das gelbes Reagenzglas
schräg das gelbe Reagenzglas
schräg das gelb Reagenzglas
schräg das grünen Reagenzglas
schräg das grüne Reagenzglas
schräg das grün Reagenzglas
schräg das große Reagenzglas
schräg das groß Reagenzglas
schräg das kleine Reagenzglas
schräg das klein Reagenzglas
schräg die ähnliche Reagenzglas
schräg die ähnlich Reagenzglas
schräg die bunte Reagenzglas
schräg die bunten Reagenzglas
schräg die bunt Reagenzglas
schräg die schwarze Reagenzglas
schräg die schwarzen Reagenzglas
schräg die schwarz Reagenzglas
schräg die weißen Reagenzglas
schräg die weiße Reagenzglas
schräg die lächelnde Rea

da  diesen blaue Reagenzglas
da  diesen blau Reagenzglas
da  diesen roten Reagenzglas
da  diesen rotes Reagenzglas
da  diesen rote Reagenzglas
da  diesen rot Reagenzglas
da  diesen zwei Reagenzglas
da  diesen drei Reagenzglas
da  diesen gelbes Reagenzglas
da  diesen gelbe Reagenzglas
da  diesen gelb Reagenzglas
da  diesen grünen Reagenzglas
da  diesen grüne Reagenzglas
da  diesen grün Reagenzglas
da  diesen große Reagenzglas
da  diesen groß Reagenzglas
da  diesen kleine Reagenzglas
da  diesen klein Reagenzglas
da  dieses ähnliche Reagenzglas
da  dieses ähnlich Reagenzglas
da  dieses bunte Reagenzglas
da  dieses bunten Reagenzglas
da  dieses bunt Reagenzglas
da  dieses schwarze Reagenzglas
da  dieses schwarzen Reagenzglas
da  dieses schwarz Reagenzglas
da  dieses weißen Reagenzglas
da  dieses weiße Reagenzglas
da  dieses lächelnde Reagenzglas
da  dieses lächelnd Reagenzglas
da  dieses blaue Reagenzglas
da  dieses blau Reagenzglas
da  dieses roten Reagenzglas
da  dieses rotes Reagenzglas

da  den bunten Reagenzglas
da  den bunt Reagenzglas
da  den schwarze Reagenzglas
da  den schwarzen Reagenzglas
da  den schwarz Reagenzglas
da  den weißen Reagenzglas
da  den weiße Reagenzglas
da  den lächelnde Reagenzglas
da  den lächelnd Reagenzglas
da  den blaue Reagenzglas
da  den blau Reagenzglas
da  den roten Reagenzglas
da  den rotes Reagenzglas
da  den rote Reagenzglas
da  den rot Reagenzglas
da  den zwei Reagenzglas
da  den drei Reagenzglas
da  den gelbes Reagenzglas
da  den gelbe Reagenzglas
da  den gelb Reagenzglas
da  den grünen Reagenzglas
da  den grüne Reagenzglas
da  den grün Reagenzglas
da  den große Reagenzglas
da  den groß Reagenzglas
da  den kleine Reagenzglas
da  den klein Reagenzglas
da  das ähnliche Reagenzglas
da  das ähnlich Reagenzglas
da  das bunte Reagenzglas
da  das bunten Reagenzglas
da  das bunt Reagenzglas
da  das schwarze Reagenzglas
da  das schwarzen Reagenzglas
da  das schwarz Reagenzglas
da  das weißen Reagenzglas
da  das weiße Reagenzglas
da  das läch

...

halbwegs dit grün Reagenzglas
halbwegs dit große Reagenzglas
halbwegs dit groß Reagenzglas
halbwegs dit kleine Reagenzglas
halbwegs dit klein Reagenzglas
halbwegs diesem ähnliche Reagenzglas
halbwegs diesem ähnlich Reagenzglas
halbwegs diesem bunte Reagenzglas
halbwegs diesem bunten Reagenzglas
halbwegs diesem bunt Reagenzglas
halbwegs diesem schwarze Reagenzglas
halbwegs diesem schwarzen Reagenzglas
halbwegs diesem schwarz Reagenzglas
halbwegs diesem weißen Reagenzglas
halbwegs diesem weiße Reagenzglas
halbwegs diesem lächelnde Reagenzglas
halbwegs diesem lächelnd Reagenzglas
halbwegs diesem blaue Reagenzglas
halbwegs diesem blau Reagenzglas
halbwegs diesem roten Reagenzglas
halbwegs diesem rotes Reagenzglas
halbwegs diesem rote Reagenzglas
halbwegs diesem rot Reagenzglas
halbwegs diesem zwei Reagenzglas
halbwegs diesem drei Reagenzglas
halbwegs diesem gelbes Reagenzglas
halbwegs diesem gelbe Reagenzglas
halbwegs diesem gelb Reagenzglas
halbwegs diesem grünen Reagenzglas
halbwegs dies

halbwegs einen schwarze Reagenzglas
halbwegs einen schwarzen Reagenzglas
halbwegs einen schwarz Reagenzglas
halbwegs einen weißen Reagenzglas
halbwegs einen weiße Reagenzglas
halbwegs einen lächelnde Reagenzglas
halbwegs einen lächelnd Reagenzglas
halbwegs einen blaue Reagenzglas
halbwegs einen blau Reagenzglas
halbwegs einen roten Reagenzglas
halbwegs einen rotes Reagenzglas
halbwegs einen rote Reagenzglas
halbwegs einen rot Reagenzglas
halbwegs einen zwei Reagenzglas
halbwegs einen drei Reagenzglas
halbwegs einen gelbes Reagenzglas
halbwegs einen gelbe Reagenzglas
halbwegs einen gelb Reagenzglas
halbwegs einen grünen Reagenzglas
halbwegs einen grüne Reagenzglas
halbwegs einen grün Reagenzglas
halbwegs einen große Reagenzglas
halbwegs einen groß Reagenzglas
halbwegs einen kleine Reagenzglas
halbwegs einen klein Reagenzglas
halbwegs einem ähnliche Reagenzglas
halbwegs einem ähnlich Reagenzglas
halbwegs einem bunte Reagenzglas
halbwegs einem bunten Reagenzglas
halbwegs einem bunt Reagen

einigermaßen etwas schwarze Reagenzglas
einigermaßen etwas schwarzen Reagenzglas
einigermaßen etwas schwarz Reagenzglas
einigermaßen etwas weißen Reagenzglas
einigermaßen etwas weiße Reagenzglas
einigermaßen etwas lächelnde Reagenzglas
einigermaßen etwas lächelnd Reagenzglas
einigermaßen etwas blaue Reagenzglas
einigermaßen etwas blau Reagenzglas
einigermaßen etwas roten Reagenzglas
einigermaßen etwas rotes Reagenzglas
einigermaßen etwas rote Reagenzglas
einigermaßen etwas rot Reagenzglas
einigermaßen etwas zwei Reagenzglas
einigermaßen etwas drei Reagenzglas
einigermaßen etwas gelbes Reagenzglas
einigermaßen etwas gelbe Reagenzglas
einigermaßen etwas gelb Reagenzglas
einigermaßen etwas grünen Reagenzglas
einigermaßen etwas grüne Reagenzglas
einigermaßen etwas grün Reagenzglas
einigermaßen etwas große Reagenzglas
einigermaßen etwas groß Reagenzglas
einigermaßen etwas kleine Reagenzglas
einigermaßen etwas klein Reagenzglas
einigermaßen dat ähnliche Reagenzglas
einigermaßen dat ähnlich R

einigermaßen dieses blau Reagenzglas
einigermaßen dieses roten Reagenzglas
einigermaßen dieses rotes Reagenzglas
einigermaßen dieses rote Reagenzglas
einigermaßen dieses rot Reagenzglas
einigermaßen dieses zwei Reagenzglas
einigermaßen dieses drei Reagenzglas
einigermaßen dieses gelbes Reagenzglas
einigermaßen dieses gelbe Reagenzglas
einigermaßen dieses gelb Reagenzglas
einigermaßen dieses grünen Reagenzglas
einigermaßen dieses grüne Reagenzglas
einigermaßen dieses grün Reagenzglas
einigermaßen dieses große Reagenzglas
einigermaßen dieses groß Reagenzglas
einigermaßen dieses kleine Reagenzglas
einigermaßen dieses klein Reagenzglas
einigermaßen diese ähnliche Reagenzglas
einigermaßen diese ähnlich Reagenzglas
einigermaßen diese bunte Reagenzglas
einigermaßen diese bunten Reagenzglas
einigermaßen diese bunt Reagenzglas
einigermaßen diese schwarze Reagenzglas
einigermaßen diese schwarzen Reagenzglas
einigermaßen diese schwarz Reagenzglas
einigermaßen diese weißen Reagenzglas
einigermaßen

einigermaßen ein klein Reagenzglas
einigermaßen dem ähnliche Reagenzglas
einigermaßen dem ähnlich Reagenzglas
einigermaßen dem bunte Reagenzglas
einigermaßen dem bunten Reagenzglas
einigermaßen dem bunt Reagenzglas
einigermaßen dem schwarze Reagenzglas
einigermaßen dem schwarzen Reagenzglas


In [ ]:
# TODO add morph parser to check for congruency, sntf transduce?
# make more complex production rules? how to join objects and their characteristics?
# (avoid sents like "das lächelnde Reagenzglas")
# OR make less complex production rules, e.g. leave adjectives out, only verb+art+object?